# Monte Carlo Simulations for Thermal Radiation Shape Factors

https://colab.research.google.com/drive/11ImDq8G1j-Y1_q2O54FVIo6ehHvp-9iD

In this notebook, I confirm the method for determining shape factor as described in "Monte Carlo Technique for the Determination of Thermal Radiation Shape Factors". I investigate the following 4 geometrical pairs: a cylinder to a plane, a plane to a cylinder, a sphere to a circle, and a circle to a sphere. As in the original paper, I choose points on the emitting surface, I choose a random cone angle and azimuth angle, and finally I examine whether such a ray would have landed on the receiving surface through simple trigonometric equations. The main difference with my current method from the original paper is that I choose points on the surface randomly rather than using an area grid. Further, I examine trade offs between time and accuracy. I also examine the efficacy of programming calculations with <i>arrays</i> rather than <i>iterative loops</i>.


<h2>Layout</h2>
To properly view the execution, blocks should be executed in order from top to bottom. (Hint: If you are new to Colab, you can run the code block by pressing the play button on the block or by pressing Shift+Enter while highlighting the block.)
<ul><li>The first block imports <i>numpy</i>, which is the package used for efficient mathematical calculations, and <i>time</i>, which is used to measure calculation times. In the first code block, constant parameters such as the locations of the plane, cylinder, and sphere are assigned.</li>
<li>The second code block contains functions used in all simulations, specifically, random number generators and a function to print results.</li>
<li>Each of the next four code blocks are four Monte Carlo simulations in the following order: 

</li>
<ul><li>Cylinder to Plane</li>
<li>Plane to Cylinder</li>
<li>Sphere to Circle</li>
<li>Circle to Sphere</li>
</ul>
</ul>
<h2>Naming Convention</h2>
In translating equations in the paper to code, here is how I named variables in the code.
<br>Symbol in Paper, Spelling in Code
<br>θ, theta
<br>φ, phi
<br>ψ, psi
<br>α, alpha
<br>β, beta
<br>γ, gamma
<br>ξ, xi
<br>E, big_e
<br>G, big_g
<br>2π-φ, psi2
<br>
<br>Functions to calculate variables are given the name <b>f_*</b>
<br>For example, a function to calculate <b>y0</b> is named <b>f_y0</b>.
<br>Functions to generate random variables are given the name <b>rand_gen_*</b>
<br>Main functions that run the Monte Carlo simulation by calling other functions are named <b>main_*</b>
<br>For example, the function to run the simulation for cylinder to plane is named <b>main_c_p</b>

#Set Parameter Constants

In [0]:
#import numpy to optimize calculations with arrays
import numpy as np
#import time to measure calculation time
from time import time

#Plane and Cylinder
params_cp={
    'x_cyl': 5.0,
    'y_cyl': 4.0,
    'r_cyl': 2.0,
    'z_min': -8.0,
    'z_max': -2.0,
    'x_plane': 0.0,
    'y_min': 0.0,
    'y_max': 8.0
}

#Theoretical Values
theory_c_p = 0.1394
theory_p_c = 0.2189

#Circle and Sphere
params_sc = {
    'x_sph': 5.0,
    'y_sph': 5.0,
    'z_sph': -5.0,
    'r_sph': 2.0,
    'x_cir': 0.0,
    'y_cir': 5.0,
    'z_cir': -5.0,
    'r_cir': 3.0
}

#Theoretical Values
theory_s_c = 0.0713
#Calculate circle to sphere shape factor by reciprocity
area_ratio = (params_sc['r_cir']**2 *np.pi)/(4*np.pi * (params_sc['r_sph']**2))
theory_c_s = theory_s_c / area_ratio


#Create Shared Functions
I use 3 types of random number generators:
<ul>
<li>A generator that produces numbers between a max and min, for example, to generate a <i>y0</i> between <i>ymin</i> and <i>ymax</i></li>
<li>A generator that produces numbers between 0 and 2pi to generate a number of angles, such as the azimuth</li>
<li>A generator that produces cone angles, using arcsin(sqrt(rand))
</ul>
<br>The print function prints the number of iterations, the result from the Monte Carlo simulation, the error, the time taken, and the calculation method used.

In [0]:
def rand_gen_max_min(shape,max_,min_):
    return np.random.random(shape) * (max_ - min_) + min_ 
def rand_gen_angle(shape):
    return np.random.random(shape) * 2 * np.pi
def rand_gen_cone(shape):
    return np.arcsin(np.sqrt(np.random.random(shape)))
def print_result_and_time(func,iterations,params,theory,method):
    iterations = int(iterations)
    t = time()
    res = func(iterations,**params)
    print('{:1.0e}\t{:.4f}\t{:1.1f}%\t{:1.2f}\t{}'
          .format(iterations,res,(res-theory)/theory*100,time()-t,method))



![alt text](https://user-images.githubusercontent.com/48815706/79038911-12daeb00-7b92-11ea-9e74-d358862243fb.png)

#Cylinder to Plane

Here, I use 2 methods. First, I run the calculation using an iterative approach. This involves generating 4 random variables, <i>z0</i> and <i>psi</i> describe a point on the cylinder and <i>theta</i> and <i>phi</i> describe the direction of a single ray. The ray is followed to the plane x=0, after which the ray's r, y, and z are calculated with the governing equations from the paper. If r is positive, and y and z are within the appropriate range, 1 is added to the count of valid rays. After the iterations, the count is divided by the number of iterations to find the shape factor.
<br><br>For a second method, random numbers for the 4 variables are generated in an array of length equal to number of iterations. Then, all calculations are done in an array form. The r, y, and z criteria output boolean arrays and simple boolean logic is used to get an array of boolean array of valid trials. The boolean array is summed and divided by iterations. Notably, the second method can be one or two orders of magnitude faster than the first.
<br><br>
<h2>Derivation of Point of Intersection</h2>

1.   <p>Generate random numbers.</p>
<p>z0 = rand * (-2--8) -8</p>
<p>theta = arcsin(sqrt(rand))</p>
<p>phi = rand * 2pi</p>
<p>psi = rand * 2pi</p>
2.   <p>Find point of emission.</p>
<p>x0 = xc-rc*sin(psi) [from paper]</p>
<p>y0 = yc-rc*cos(psi) [from paper]</p>
<p>z0 given above</p>
3.   <p>Follow ray to target plane.</p>
<p>x = 0</p>
<p>x = x0 + r*cos(theta)*sin(2pi-psi) + r*sin(theta)*cos(E)*cos(2pi-psi) [from paper]</p>
<p>r = (x-x0) / [cos(theta)*sin(2pi-psi) + r*sin(theta)*cos(E)*cos(2pi-psi)]</p>
<p>r must be positive. A negative r represents a ray that has extended in the direction opposite the plane.
4.  <p>See if resulting y and z points are between the ymin and ymax and zmin and zmax, respectively.</p>
<p><img width=300px src="https://user-images.githubusercontent.com/48815706/79050333-2d3ab600-7bde-11ea-8559-cc8c185b2c2b.JPG">[from paper]</p><p>See if y is between 0 and 8.</p><p>See if z is between -8 and -2.</p>




In [0]:
#psi2 is 2pi - psi
def f_psi2(psi):
    return 2 * np.pi - psi
def f_big_e(phi,psi,array=True):
    if array:
        return f_big_e_array(phi,psi)
    return phi if psi <= np.pi else phi + np.pi
#When running array calculation, use f_big_e_array instead of f_big_e
def f_big_e_array(phi,psi):
    big_e = phi.copy()
    I = psi > np.pi
    big_e[I] = np.pi + big_e[I]
    return big_e
def f_x0(r_cyl,x_cyl,psi):
    return x_cyl - r_cyl * np.sin(psi)
def f_y0(r_cyl,y_cyl,psi):
    return y_cyl - r_cyl * np.cos(psi)
#NOTE! if emission never touches x0, function will likely return very large number, or possibly nan
#NOTE! r may be negative, which should be filtered as no touch
def f_r(x0,x_plane,theta,phi,psi,array=False):
    psi_2 = f_psi2(psi)
    big_e = f_big_e(phi,psi,array=array)
    return (x_plane-x0)/(
        (np.cos(theta) * np.sin(psi_2)) + 
        (np.sin(theta) * np.cos(big_e)*np.cos(psi_2))
        )
def f_y(r,y0,theta,phi,psi,array=False):
    psi_2 = f_psi2(psi)
    big_e = f_big_e(phi,psi,array=array)
    return y0 - r *np.cos(theta)* np.cos(psi_2) + r * np.sin(theta) * np.cos(big_e) * np.sin(psi_2)
def f_z(r,z0,theta,phi):
    return z0 - r* np.sin(theta) * np.sin(phi)
def f_calc_ryz(r_cyl,x_cyl,y_cyl,x_plane,z0, theta,phi,psi,array=False):
    x0= f_x0(r_cyl,x_cyl,psi)
    y0 = f_y0(r_cyl,y_cyl,psi)
    r = f_r(x0,x_plane,theta,phi,psi,array=array)
    y = f_y(r,y0,theta,phi,psi,array=array)
    z = f_z(r,z0,theta,phi)
    return r,y,z

#Traditional Iteration:
def main_c_p(num_iterations,r_cyl,x_cyl,y_cyl,x_plane,y_max,y_min,z_max,z_min):
    count=0
    shape = ()
    for _ in range(num_iterations):
        r,y,z = f_calc_ryz(
            r_cyl = r_cyl,
            x_cyl = x_cyl,
            y_cyl = y_cyl,
            x_plane=x_plane,
            z0 = rand_gen_max_min(shape,z_max,z_min),
            theta = rand_gen_cone(shape),
            phi = rand_gen_angle(shape),
            psi = rand_gen_angle(shape),
            array=False
            )
        #SKIP if r < 0. It means ray goes opposite direction of plane
        if r < 0:
            continue
        if y >= y_min:
            if y <= y_max:
                if z >= z_min:
                    if z<=z_max:
                        count+=1
    return count / num_iterations

# Note: Running calculations in an array, we an run the simulation an order of magnitude faster
def main_c_p_array(num_iterations,r_cyl,x_cyl,y_cyl,x_plane,y_max,y_min,z_max,z_min):
    shape = (num_iterations,)
    r,y,z = f_calc_ryz(
            r_cyl = r_cyl,
            x_cyl = x_cyl,
            y_cyl = y_cyl,
            x_plane = x_plane,
            z0 = rand_gen_max_min(shape,z_max,z_min),
            theta = rand_gen_cone(shape),
            phi = rand_gen_angle(shape),
            psi = rand_gen_angle(shape),
            array=True
            )
    y_pass = np.logical_and(y>=y_min, y<=y_max)
    z_pass = np.logical_and(z>=z_min,z<=z_max)
    yz_pass = np.logical_and(y_pass,z_pass)
    r_pass = np.logical_and(yz_pass,r>0)
    
    return np.sum(r_pass) / num_iterations
print('Cylinder to Plane, Theoretical:{:.4f}'.format(theory_c_p))
print('\nTrials\tResult\tError\tTime\tMethod')
for iterations in [1e2,1e3,1e4,1e5,1e6]:
    print_result_and_time(main_c_p,iterations,params_cp,theory_c_p,method='Iteration')
    print_result_and_time(main_c_p_array,iterations,params_cp,theory_c_p,method='Array')
    print('-'*40)

Cylinder to Plane, Theoretical:0.1394

Trials	Result	Error	Time	Method
1e+02	0.1200	-13.9%	0.00	Iteration
1e+02	0.1900	36.3%	0.00	Array
----------------------------------------
1e+03	0.1260	-9.6%	0.04	Iteration
1e+03	0.1510	8.3%	0.00	Array
----------------------------------------
1e+04	0.1403	0.6%	0.40	Iteration
1e+04	0.1364	-2.2%	0.01	Array
----------------------------------------
1e+05	0.1392	-0.2%	3.92	Iteration
1e+05	0.1391	-0.2%	0.05	Array
----------------------------------------
1e+06	0.1387	-0.5%	38.88	Iteration
1e+06	0.1394	-0.0%	0.54	Array
----------------------------------------


Here, it is obvious that using the array method is much faster than using a loop. From this point on, all simulations will be done using the array method. Also, with only 10,000 trials, the error is only about 2.2%, with only minor improvements when increasing trials by factors of ten. Still, the computer can calculate 1,000,000 trials in only 0.54 seconds, so there is no downside to having this many trials.


# Plane to Cylinder
While the plane-to-cylinder shape factor can be obtained with previous results and the reciprocity rule, in the interest of the Monte Carlo technique as a robust solution, I use the technique again for plane to cylinder.
<h2>Derivation of Point of Intersection</h2>


1.   Generate random variables for emission from plane.
<p>y0 = rand * 8</p>
<p>z0 = rand * (-2--8)-8</p>
<p>theta = arcsin(sqrt(rand))</p>
<p>phi = rand * 2pi</p>
2.   Calculate possible intersection with cylinder.
<p><img src="https://user-images.githubusercontent.com/48815706/79050801-46913180-7be1-11ea-822a-12308e65a95a.jpg"> [from paper]</p>
<p>The value under the radical must be positive.</p>
<p>r must be positive.</p>
3. See if z is between zmin and zmax.
<p><img src="https://user-images.githubusercontent.com/48815706/79050908-e6e75600-7be1-11ea-839e-78fa6e1b3cda.jpg"> [from paper]</p>
<p>See if z is between -8 and -2.</p>




In [7]:
def f_a(theta,phi):
    return np.square(np.cos(theta))+np.square(np.sin(theta))*np.square(np.cos(phi))
def f_b(theta,phi,y0,x_cyl,y_cyl):
    return 2*y_cyl*np.sin(theta)*np.cos(phi) - 2*x_cyl*np.cos(theta) - 2*y0*np.sin(theta)*np.cos(phi)
def f_c(y0,r_cyl,x_cyl,y_cyl):
    return np.square(y_cyl) + np.square(x_cyl) + np.square(y_cyl) - 2*y_cyl*y0 - np.square(r_cyl)
#Outputs r and an boolean array which is True where r>0
def f_r_and_valid(a,b,c):
    r = np.zeros_like(a)
    #check if negative under radical
    I = (4*a*c)>(np.square(b))
    r[~I]= ((-1* b[~I] - np.sqrt(np.square(b[~I])-4*a[~I]*c[~I]))/(2*a[~I])) 
    return r, r>(1e-10)
def f_z(z0,r,theta,phi):
    return z0 + r * np.sin(theta)*np.sin(phi)
def valid_intersection(y0,z0,r_cyl,x_cyl,y_cyl,z_max,z_min,theta,phi):
    a = f_a(theta,phi)
    b = f_b(theta,phi,y0,x_cyl,y_cyl)
    c = f_c(y0,r_cyl,x_cyl,y_cyl)
    r,valids = f_r_and_valid(a,b,c)
    z = f_z(z0,r,theta,phi)
    return np.logical_and(valids,np.logical_and(z>=z_min,z<=z_max))
def rand_gen_theta(shape):
    return np.arcsin(np.sqrt(np.random.random(shape)))
def rand_gen_phi(shape):
    return np.random.random(shape) * 2 * np.pi
def rand_gen_z0(shape,zmax,zmin):
    return np.random.random(shape) * (zmax-zmin) +zmin
def rand_gen_y0(shape,ymax,ymin):
    return np.random.random(shape) * (ymax-ymin) +ymin
def main_p_c(iterations,r_cyl,x_cyl,y_cyl,x_plane,y_max,y_min,z_max,z_min):
    shape = (iterations,)
    y0 = rand_gen_max_min(shape,y_max,y_min)
    z0 = rand_gen_max_min(shape,z_max,z_min)
    theta = rand_gen_cone(shape)
    phi = rand_gen_angle(shape)
    res= valid_intersection(y0,z0,r_cyl,x_cyl,y_cyl,z_max,z_min,theta,phi)
    return np.sum(res) / iterations

print('Plane to Cylinder, Theoretical:{:.4f}'.format(theory_p_c))
print('\nTrials\tResult\tError\tTime\tMethod')
for iterations in [1e2,1e3,1e4,1e5,1e6,1e7]:
    print_result_and_time(main_p_c,iterations,params_cp,theory_p_c,method='Array')   

Plane to Cylinder, Theoretical:0.2189

Trials	Result	Error	Time	Method
1e+02	0.2400	9.6%	0.00	Array
1e+03	0.2000	-8.6%	0.00	Array
1e+04	0.2211	1.0%	0.01	Array
1e+05	0.2175	-0.6%	0.04	Array
1e+06	0.2159	-1.3%	0.40	Array
1e+07	0.2159	-1.4%	3.82	Array


Interestingly, the model seems to generate fairly accurate results from 10,000 trials on, similar to in the cylinder-to-plane simulations. At 10,000 trials, the error is 1.0%, while results with larger trial sizes are -0.6%, -1.3%, and -1.4%. The accuracy seems to fluctuate with added trials - rather than become more accurate. This may be due to the randomness inherent in Monte Carlo simulations. I may then hypothesize that the 10,000 was "lucky" and that with re-runs of the simulation, larger trial sizes should produce more <i>precise</i> results (i.e. results with deviation). To test this, I run the simulations again with multiple runs of each trial size from 1e4 to 1e7.

In [9]:
runs = 5
print('Plane to Cylinder (Multiple Runs with Same Trial Size), Theoretical:{:.4f}'.format(theory_p_c))
print('\nTrials\tResult\tError\tTime\tMethod')
for iterations in [1e4,1e5,1e6,1e7]:
    for _ in range(runs):
        print_result_and_time(main_p_c,iterations,params_cp,theory_p_c,method='Array')   
    print('-'*40)

Plane to Cylinder (Multiple Runs with Same Trial Size), Theoretical:0.2189

Trials	Result	Error	Time	Method
1e+04	0.2109	-3.7%	0.01	Array
1e+04	0.2213	1.1%	0.01	Array
1e+04	0.2083	-4.8%	0.00	Array
1e+04	0.2118	-3.2%	0.00	Array
1e+04	0.2150	-1.8%	0.00	Array
----------------------------------------
1e+05	0.2143	-2.1%	0.05	Array
1e+05	0.2160	-1.3%	0.05	Array
1e+05	0.2163	-1.2%	0.05	Array
1e+05	0.2153	-1.7%	0.05	Array
1e+05	0.2162	-1.2%	0.05	Array
----------------------------------------
1e+06	0.2167	-1.0%	0.42	Array
1e+06	0.2153	-1.6%	0.42	Array
1e+06	0.2155	-1.5%	0.42	Array
1e+06	0.2158	-1.4%	0.42	Array
1e+06	0.2155	-1.5%	0.41	Array
----------------------------------------
1e+07	0.2160	-1.3%	3.85	Array
1e+07	0.2160	-1.3%	3.87	Array
1e+07	0.2160	-1.3%	3.90	Array
1e+07	0.2160	-1.3%	3.89	Array
1e+07	0.2160	-1.3%	3.89	Array
----------------------------------------


Testing the trial size versus precision hypothesis, results seem to validate the idea that more trials can give a more reliable shape factor. Note how with 1e4 trials, the error is between -4.8% and 1.1% on five runs. With 1e5 trials, it's between -2.1% and -1.2%, with 1e6 trials, it's between -1.6% and -1.0%, and with 1e7 trials, it's consistently at -1.3%. Strangely, the results seem to favor a negative error. Perhaps the theoretical number has been rounded up on its last decimal. Perhaps the float precision in the code execution is making errors when trying to determine inequalities near edges. For example, with floats, if two values are meant to be equal (like 0.01 and 0.00999999), they may fall toward the wrong side due to how the underlying code deals with precision rounding.

<p><img src="https://user-images.githubusercontent.com/48815706/79054472-f2df1200-7bf9-11ea-9451-70f221ffa73a.jpg"></p>


# Sphere to Circle
Here, I follow the same procedure as in the previous simulations, but with new geometries.

<h2>Derivation of Point of Intersection</h2>





1.   Generate random variables.<p>x0 = rand * (xmax-xmin) + xmin, where xmin and xmax are boundaries of the sphere</p>
<p>gamma = rand * 2pi</p>
<p>theta = arcsin(sqrt(rand))</p>
<p>phi = rand * 2pi</p>
2.   Calculate the emission point.
<p><img src="https://user-images.githubusercontent.com/48815706/79052447-79402780-7beb-11ea-83f8-e5a6c347f3fc.jpg"> [from paper]</p>
<p><img src="https://user-images.githubusercontent.com/48815706/79052511-da67fb00-7beb-11ea-8460-6e466ccd44e2.jpg"></p>
<p>alpha = arccos((x0-x_sph)/r_sph)</p>
<p>The figure below shows a 2-dimensional view of 1000 points generated on the outside of the sphere.</p>
<p><img src="https://user-images.githubusercontent.com/48815706/79053044-28cac900-7bef-11ea-8cb6-3180e915ae2f.png"></p>
3.   Follow the ray to the plane x=0 (on which the circle lies).
<p><img src="https://user-images.githubusercontent.com/48815706/79052557-421e4600-7bec-11ea-9390-edb7bb1ee0ea.jpg"> [from paper]</p>
<p>r = (x_cir-x0)/[cos(theta)*cos(beta)*cos(G)-sin(theta)*sin(phi)*sin(G)+sin(theta)*cos(phi)*sin(beta)*cos(G)]</p>
<p>r must be positive.</p>

4.  See if y and z are in the circle on plane x=0.
<p><img src="https://user-images.githubusercontent.com/48815706/79052650-dab4c600-7bec-11ea-88f9-17304131ca7e.jpg"> [from paper]</p>
<p> See if y and z points are within circle using equation:<br>
(y-y_cir)^2 + (z-z_cir)^2 <= r_cir ^2
</p>


In [13]:
def f_y0(r_sph,y_sph,alpha,gamma):
    return y_sph + r_sph*np.sin(alpha)*np.cos(gamma)
def f_z0(r_sph,z_sph,alpha,gamma):
    return z_sph - r_sph*np.sin(alpha)*np.sin(gamma)
def f_beta(y0,r_sph,y_sph): 
    return np.arcsin((y0-y_sph)/r_sph)
def f_alpha(r_sph,x_sph,x0):
    return np.arccos((x0-x_sph)/r_sph)
def f_xi(x0,z0,x_sph,z_sph):
    return np.arctan((z_sph-z0)/(x0-x_sph))
def f_big_g(x0,z0,x_sph,z_sph,xi):
    big_g = xi.copy()
    I = np.logical_and(z0<z_sph,x0<x_sph)
    big_g[I]=big_g[I]+np.pi
    I = np.logical_and(z0>z_sph,x0<x_sph)
    big_g[I]=big_g[I]+np.pi
    I = np.logical_and(z0>z_sph,x0>x_sph)
    big_g[I]=big_g[I]+2*np.pi
    I = np.logical_and(z0==z_sph,x0==x_sph)
    big_g[I]=np.pi/2
    return big_g
def f_y(r,y0,theta,phi,beta):
    return y0 + r*np.cos(theta)*np.sin(beta)-r*np.sin(theta)*np.cos(phi)*np.cos(beta)
def f_z(r,z0,theta,phi,beta,big_g):
    return z0-r*np.cos(theta)*np.cos(beta)*np.sin(big_g)-r*np.sin(theta)*np.sin(phi)*np.cos(big_g)-r*np.sin(theta)*np.cos(phi)*np.sin(beta)*np.sin(big_g)
def f_r(x0,x_cir,theta,phi,beta,big_g):
    return (x_cir-x0)/(np.cos(theta)*np.cos(beta)*np.cos(big_g)-np.sin(theta)*np.sin(phi)*np.sin(big_g)+np.sin(theta)*np.cos(phi)*np.sin(beta)*np.cos(big_g))

def rand_gen_gamma(shape):
    return rand_gen_generic_angle(shape)
def check_circle(y,z,r_cir,y_cir,z_cir):
    return (np.square(y-y_cir) + np.square(z-z_cir))<np.square(r_cir)

def main_s_c(iterations,r_sph,x_sph,y_sph,z_sph,r_cir,x_cir,y_cir,z_cir):
    xmax = x_sph+r_sph
    xmin = x_sph-r_sph
    
    shape = (iterations,)
    x0 =rand_gen_max_min(shape,xmax,xmin) 
    gamma = rand_gen_angle(shape)
    theta = rand_gen_cone(shape)
    phi = rand_gen_angle(shape)
    
    alpha = f_alpha(r_sph,x_sph,x0)
    y0 = f_y0(r_sph,y_sph,alpha,gamma)
    z0 = f_z0(r_sph,z_sph,alpha,gamma)
    beta = f_beta(y0,r_sph,y_sph)
    xi = f_xi(x0,z0,x_sph,z_sph)
    big_g = f_big_g(x0,z0,x_sph,z_sph,xi)

    r = f_r(x0,x_cir,theta,phi,beta,big_g)
    y = f_y(r,y0,theta,phi,beta)
    z = f_z(r,z0,theta,phi,beta,big_g)
    #r must be positive and y,z must fall inside of the circle
    yz_pass = check_circle(y,z,r_cir,y_cir,z_cir)
    r_pass = np.logical_and(yz_pass,r>0)
    return np.sum(r_pass) / iterations
print('Sphere to Circle, Theoretical:{:.4f}'.format(theory_s_c))
print('\nTrials\tResult\tError\tTime\tMethod')
for iterations in [1e2,1e3,1e4,1e5,1e6,1e7]:
    print_result_and_time(main_s_c,iterations,params_sc,theory_s_c,method='Array')  

Sphere to Circle, Theoretical:0.0713

Trials	Result	Error	Time	Method
1e+02	0.0400	-43.9%	0.00	Array
1e+03	0.0930	30.4%	0.00	Array
1e+04	0.0740	3.8%	0.01	Array
1e+05	0.0726	1.8%	0.12	Array
1e+06	0.0713	0.1%	1.08	Array
1e+07	0.0712	-0.1%	10.28	Array


Sphere-to-circle results are as expected. With increasing trial size, accuracy improves. At 1,000,000 trials, the error is 0.1%, and it is calculated in 1.08 seconds. This is outstanding!

# Circle to Sphere
The final 4th environment is the radition from the circle on plane x=0 to the sphere. The procedure should be well understood by now.
<h2>Derivation of Point of Intersection</h2>


1.   Generate random variables.
<p>r0= sqrt(rand) * r_cir</p>
<p>angle0 = rand * 2pi</p>
<p>theta = arcsin(sqrt(rand))</p>
<p>phi = rand * 2pi</p>
2.   Calculate the emission point.
<p><img src="https://user-images.githubusercontent.com/48815706/79053645-bdcfc100-7bf3-11ea-80eb-d5bd76bac2e2.jpg"></p>
<p>x0 = 0</p>
<p>y0 = y_cir + r0*sin(angle0)</p>
<p>z0 = z_cir - r0*cos(angle0)</p>
<p>The figure below shows 1000 randomly generate points on the x=0 plane.</p>
<p><img src="https://user-images.githubusercontent.com/48815706/79053926-e5c02400-7bf5-11ea-8e5f-a03cc2c33118.png"></p>
3. Calculate possible intersection with sphere.
<p><img src="https://user-images.githubusercontent.com/48815706/79053798-cecd0200-7bf4-11ea-8b19-15f6083597aa.jpg"> [from paper]</p>
<p> Value under radical must not be negative. </p>
<p> r must be positive.</p>


In [15]:
def f_a():
    return 1.0
def f_b(y0,z0,x_sph,y_sph,z_sph,theta,phi):
    return np.sin(theta) *np.cos(phi)*(2*y_sph-2*y0)+np.sin(theta)*np.sin(phi)*(2*z0-2*z_sph)-2*x_sph*np.cos(theta)
def f_c(y0,z0,r_sph,x_sph,y_sph,z_sph):
    return np.square(x_sph)+np.square(y_sph)+np.square(z_sph)+np.square(y0)+np.square(z0)-2*y_sph*y0-2*z_sph*z0-np.square(r_sph)
#Outputs r and an boolean array which is True where r>0
def f_r_and_valid(a,b,c):
    r = np.zeros_like(b)
    #check if negative under radical
    I = (4*a*c)>(np.square(b))
    r[~I]= ((-1* b[~I] - np.sqrt(np.square(b[~I])-4*a*c[~I]))/(2*a)) 
    return r, r>(1e-10)
def f_y0(r0,y_cir,angle0):
    return y_cir + r0 * np.sin(angle0)
def f_z0(r0,z_cir,angle0):
    return z_cir - r0 * np.cos(angle0)
def valid_intersection(y0,z0,r_sph,x_sph,y_sph,z_sph,theta,phi):
    a = f_a()
    b = f_b(y0,z0,x_sph,y_sph,z_sph,theta,phi)
    c = f_c(y0,z0,r_sph,x_sph,y_sph,z_sph)
    r,valids = f_r_and_valid(a,b,c)
    return valids
def rand_gen_theta(shape):
    return np.arcsin(np.sqrt(np.random.random(shape)))
def rand_gen_phi(shape):
    return np.random.random(shape) * 2 * np.pi
def rand_gen_r0(shape,r_cir):
    return np.sqrt(np.random.random(shape)) * r_cir
def rand_gen_angle0(shape):
    return np.random.random(shape) *2 * np.pi
def main_c_s(iterations,r_sph,x_sph,y_sph,z_sph,r_cir,x_cir,y_cir,z_cir):
    shape = (iterations,)
    r0 = rand_gen_r0(shape,r_cir)
    angle0 = rand_gen_angle(shape)
    y0 = f_y0(r0,y_cir,angle0)
    z0 = f_z0(r0,z_cir,angle0)
    theta = rand_gen_cone(shape)
    phi = rand_gen_angle(shape)
    res= valid_intersection(y0,z0,r_sph,x_sph,y_sph,z_sph,theta,phi)
    return np.sum(res) / iterations
print('Circle to Sphere, Theoretical:{:.4f}'.format(theory_c_s))
print('\nTrials\tResult\tError\tTime\tMethod')
for iterations in [1e2,1e3,1e4,1e5,1e6,1e7]:
    print_result_and_time(main_c_s,iterations,params_sc,theory_c_s,method='Array')  

Circle to Sphere, Theoretical:0.1268

Trials	Result	Error	Time	Method
1e+02	0.1200	-5.3%	0.00	Array
1e+03	0.1490	17.5%	0.00	Array
1e+04	0.1257	-0.8%	0.01	Array
1e+05	0.1271	0.3%	0.04	Array
1e+06	0.1268	0.1%	0.35	Array
1e+07	0.1267	-0.0%	3.35	Array


The trial size 100 had a low error at -5.3%. This was probably "lucky" as demonstrated with multiple runs in the plane-to-cylinder block. Other than that, accuracy increases with trial size. At 1,000,000 trials, the calculated shape factor is about 0.1% off the theoretical.

#Summary
Below is a summary of calculated shape factors at 1,000,000 samples, along with theoretical values and error. (Shape factors are rounded to the 4th decimal place, which is why errors can be 0.1%, though the values appear to match the theoreticals.)


*   Cylinder to Plane SF: 0.1394 TH:0.1394 ERR: -0.0%
*   Plane to Cylinder SF: 0.2159 TH: 0.2189	ERR: -1.3%
* Sphere to Circle SF: 0.0713	TH: 0.0713 ERR: 0.1%
* Circle to Sphere SF: 0.1268	TH: 0.1268 ERR: 0.1% 

<p>Overall, the largest error is -1.3%, while the rest are very small. Likely modern technology, including computing power, random number generators, and optimized calculation has allowed this experiment to surpass the accuracy of the studied paper's. Moreover, these shape factor calculations can be run in a few seconds or less on a fair laptop in 2020. The Monte Carlo technique therefore seems invaluable for real world problems.</p>


